# How to mafia 101
    Data scraping with Python
    By: Javier
    
### Requirements:
- Use python3 to avoid text encoding problems.
- Some extra libraries: requests (request websites), selenium (to control firefox), beautifulsoup (parse html), json (parse json), fake_headers...
- Firefox and geckodriver (https://github.com/mozilla/geckodriver/releases) to control with selenium
- Jupyter notebook (autocomplete, interactive, great). Anacondas provides everything.

### Every data service has an API
- Some sites allow you to use it directly (Twitter) -> Easy
- Some sites hide it (most websites) -> Easyish
- Some sites block it (most apps and services whose income come from data) -> No idea

In [ ]:
!pip install tweepy
!pip install fake_useragent #this will allow us to use random headers, many websites track it
!pip install selenium --upgrade

In [1]:
#Imports
import pickle
import time
from IPython.core.display import display, Image

# 1. Types of data scraping
## 1.1 Using the API that the site provides (e.g. Twitter). 
- Usually you send a request and the site gives you back json (a dictionary with the data).
- Chances are that somebody else have coded a python package for you.
- They have limits.

Example: Twitter
- Get a key: https://apps.twitter.com/
- Documentation: https://dev.twitter.com/rest/public
- Find a library: https://dev.twitter.com/resources/twitter-libraries
(let's use https://github.com/tweepy/tweepy)


Limitations
- 100 mess/query
- 180 messages every 15 min
- Only one week of data
- Not mafia (unless many accounts, then very mafia)

In [117]:
def twitter(d_keys,query):
    import tweepy
    import time

    #Authtentify
    auth = tweepy.OAuthHandler(d_keys["CONSUMER_KEY"], d_keys["CONSUMER_SECRET"])
    auth.set_access_token(d_keys["ACCESS_KEY"], d_keys["ACCESS_SECRET"])
    api = tweepy.API(auth)

    #We want 1000 tweets
    num_results = 1000
    result_count = 0
    last_id = None
    
    #Max 180 tweets 15 min
    cumulative = 0

    #While we don't have them
    while (result_count <  num_results):
        previous_tweets = result_count
        #Ask for more tweets, starting in the last_id (identifier of the tweet)
        results = api.search(q = query,
                              count = 90, max_id = last_id,result_type="recent")
                                # geocode = "{},{},{}km".format(latitude, longitude, max_range) #for geocode

        #for each tweet extract some info (json structure)
        for result in results:
            result_count += 1
            user = result.user.screen_name
            text = result.text
            followers_count = result.user.followers_count
            time_zone = result.user.time_zone
            print("_"*10)
            print(user,time_zone,followers_count)
            print(text)

        #keep the last_id to know where to continue
        last_id = int(result.id)-1
        new_tweets = result_count - previous_tweets

        print ("Number of results: {} ({} new)".format(result_count,new_tweets))

        #If we don't get new tweets exit
        if new_tweets == 0: 
            break
        
        time.sleep(1)
        
        if ((result_count + 90) // 150) > cumulative:
            cumulative += 1
            time.sleep(15*60)


d_keys = pickle.load(open(".key","rb")) #don't share your keys ;)
twitter(d_keys,"from:wwcs_2017")

__________
wwcs_2017 None 55
At WWCS2017 in @is_petnica we will also discuss "Scientific Careers in Academia and Beyond": https://t.co/d8YNNilt9A ask your qstns #wwcs17
__________
wwcs_2017 None 55
Full detailed program of @wwcs_2017 can be found here: https://t.co/EZjejUrM80 https://t.co/OuzBL5TiaZ
__________
wwcs_2017 None 55
RT @YRN_CS: Bridge Grants 2017 to foster interdisciplinary research efforts: waiting for great proposals! Deadline is March 5th! https://t.…
__________
wwcs_2017 None 55
Our program begins this sunday with the Belgrade city tour! Looking forward to meeting you all!!!
Number of results: 4 (4 new)
Number of results: 4 (0 new)


## 1.2 Using the hidden API
- You request the website and get **json/html** back.
- You need to find patterns in their API (may be hard sometimes).
- No limits usually. 
    - Try to behave, don't get confused with a DDoS attack.
    - Check the terms of service of the website.
    - I got UvA blocked from a site last year.
- It can be automatized to run periodically: https://scrapy.org/
- Very mafia

What will we use:
- The inspector and network tab in the broser
- This web to convert curl queries to python request queries: https://curl.trillworks.com/


#### 1.2.1 UVM example
- Info about students: https://www.uvm.edu/directory/

In [120]:
def get_student_info(netIDs):
    """
    Calculate how many students of first,second,third year are in our database
    """
    
    import requests
    import json
    from collections import Counter
    
    ids = []
    n = []
    for ID in netIDs:
        #Request
        r = requests.get('https://www.uvm.edu/directory/api/query_results.php?name={0}&department=&phone=&soundslike=0&request_num=1'.format(ID.replace(".","+") ))
        
        #Get the json
        try: 
            t = json.loads(r.text)
            n.append(t["data"][0]["ou"]["0"])
            ids.append(ID)
        except: 
            print("Error with ",ID)
    
    print(ids)
    print(n)
    print(Counter(n))



In [ ]:
netIDs = ['hhoffma2', 'nswright', 'emily.sippin', 'emily.sippin', 'cjmarsha', 'nicholas.grubinger', 'kabdi', 'wbourne', 'hdeng', 'caroline.hooper', 'mghafoor', 'cecappel', 'oolafarg', 'cyang3', 'wgarratt', 'imchale', 'mgalla13', 'tdwise', 'jgterino', 'escott3', 'ailer', 'sarah.mantz', 'mleskova', 'bfilker1', 'jpconsta', 'amcgrory', 'verhoeff.parker', 'aharte', 'emcwrigh', 'dgrundha', 'amanda.silva', 'ajstone', 'dgrundha', 'evolaj', 'acoonwil', 'lberelso', 'solmaz.karimi', 'jmorri18', 'bcpincus', 'faliaj', 'iroach', 'taylor.bird', 'rnesnevi', 'lmyers4', 'mgallag9', 'agryan', 'meschnei', 'rfcalabr', 'tlo1', 'asulli19', 'Bcrocke1', 'wocarrol', 'kkaiters', 'wsahene', 'bsalimi', 'hhoffma2', 'ndudkina', 'pgreenfi', 'grace.jia', 'cswinn', 'tmheffer', 'stosto', 'stosto', 'amcgrory', 'jfiggie', 'uacharya', 'amanda.silva', 'ebambury', 'swshelto', 'mprogers', 'ebambury', 'kgonski', 'ebambury', 'agryan', 'ashah7', 'jbenelli', 'nwoodcoc', 'ejsulliv', 'jldickin', 'aculupa', 'jbfrankl', 'mprogers', 'rjplatt', 'jbfrankl', 'mghafoor', 'swbreen', 'swbreen', 'gprodans', 'amguy', 'amguy', 'vavalone', 'gcongius', 'jwaksman', 'rlangdon', 'c.dong', 'c.dong', 'sphuong', 'rfranken', 'smatika']
get_student_info(netIDs)

##### 1.2.2 UVM example 2
- Donwload all the ISBNs of the books that will be used
- http://uvmbookstore.uvm.edu/textbook_express.asp?mode=2&step=2


In [123]:
def extractBooks():
    """
    Get all the books used at UVM in one year
    """
    import requests
    from bs4 import BeautifulSoup

    cookies = {
        's_vnum': '1440621181017%26vn%3D1',
        's_fid': '0EF2E6D54E22CAAA-084C34FDA4EC5F3D',
        '__utma': '54619247.234461063.1437860582.1438191847.1438191847.1',
        '__utmz': '54619247.1438191847.1.1.utmcsr=google|utmccn=(organic)|utmcmd=organic|utmctr=(not%20provided)',
        'ASPSESSIONIDSCQDDRCQ': 'FBDNKLMABLDHPHCHGANLMKCG',
        'ASPSESSIONIDQCSBCQDT': 'OEDPCJMAMCFMELHHNLCMKLHG',
        'referring_url': 'https%3A//www.google.es/',
        '_ga': 'GA1.2.234461063.1437860582',
        '_gat': '1',
        '_gat_unitTracker': '1',
        'mscssid': '280FE7B7BA944614A67CE36A236CBAE3',
        'cookies': 'true',
    }

    headers = {
        'Pragma': 'no-cache',
        'Origin': 'http://uvmbookstore.uvm.edu',
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language': 'es,en-US;q=0.8,en;q=0.6',
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.89 Safari/537.36',
        'HTTPS': '1',
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Cache-Control': 'no-cache',
        'Referer': 'http://uvmbookstore.uvm.edu/default.asp?',
        'Connection': 'keep-alive',
    }

    #All the books start with "70 and then four numbers". Let's try all the combinations
    books = [ "7"+"0"*(4-len(str(_)))+str(_) for _ in range(0,1+9999)]

    while books:
        #keep the top 100
        booksT = books[:100]
        books = books[100:]
        
        #join the ids of the books
        booksT = "%2C".join(booksT)
    
        #create the data and post it
        data = 'tbe-block-mode=0&selTerm=0%7C0&generate-book-list=Get+Your+Books&sectionIds='+booksT
        r = requests.post('http://uvmbookstore.uvm.edu/textbook_express.asp?mode=2&step=2', headers=headers, cookies=cookies, data=data)
        #get the html and parse the html
        html = r.text
        soup = BeautifulSoup(html)

        #find all the isbns
        ids = soup.find_all('span', {'class' : 'isbn'})

        #Print them
        for _ in ids:
            print(_.text)
        

In [124]:
extractBooks()

True


/home/javiergb/Programs/anaconda3/lib/python3.5/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 184 of the file /home/javiergb/Programs/anaconda3/lib/python3.5/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


013035256X
1783600225
1626370311
073605250X
1400053099
0547406274
0077595114
1284026337
0133864960
0393912787
1581211279
1433805618
0470454989
1579229840
1579225845
0803626827
1455741655
0226816389
1422485560
0073525308
113359414X
0140443622
0199537380
0872207234
0199537569
0199730571
1590706994
0078112761
1581212151
1563683008
158121135X
158121054X
0199777543
0770435688
0634038168
0312552017
1557783926
0199931925
1889293164
0205998259


KeyboardInterrupt: 

## 1.3 Scrapping easy websites
- You request the website and get **html** back.
- You need to find patterns to extract the data (not too hard, but takes a bit of time).
- No limits usually. 
    - Try to behave, don't get confused with a DDoS attack.
    - Check the terms of service of the website.
    - I got UvA blocked from a site last year.
- It can be automatized to run periodically: https://scrapy.org/


- You can download files as well (urlrequest package).

In [125]:
from fake_useragent import UserAgent
import json
import re
import bs4

ua = UserAgent()

def login():
    """
    Use the api to log in
    """
    res = requests.post('https://bookscouter.com/login.php', 
                        data = {'action': 'login','ref':'','email':'7320939@etlgr.com','pass':'wwcs'})
    print(res.ok)
    
def get_price_bookscouter(ISBN):
    """
    Request the website with random headers, extract the table
    """
    #Get html
    url = 'http://bookscouter.com/historic/'+ISBN
    res = requests.get(url, headers={'User-Agent': '%s' %ua.random})
    
    #Check it's all good
    if res.status_code != 200: 
        print("Error code {}".format(es.status_code))
        return ""

    #Parse html
    soup = bs4.BeautifulSoup(res.text)
    
    #Get table
    prices = soup.find_all('td')
    
    #Return last price
    if len(prices) > 0:
        return float(prices[2].text.replace('$', ''))

    else:
        return None


In [126]:
login() #use the hidden api to log in
get_price_bookscouter("978-1491910290") #get the book

True


4.13

## 1.4 Behaving like a person (never a bad idea)
- It's slow
- It can handle javascript
- You get **html** code back.
- Behave.


- Requirements (one):
    - Firefox + geckodriver (https://github.com/mozilla/geckodriver/releases_
    - Chrome + chromedriver
    
    
    geckodriver/chromedriver must be execution permissions (chmod +x geckodriver)



In [91]:
import selenium.common.exceptions
import selenium.webdriver
import selenium.webdriver.common.desired_capabilities
import selenium.webdriver.support.ui
from selenium.webdriver.support import expected_conditions

#Define a function to wait for an element to load
def _wait_for_element(xpath, wait):
    try:
        polling_f = expected_conditions.presence_of_element_located((selenium.webdriver.common.by.By.XPATH, xpath))
        elem = wait.until(polling_f)
    except: 
        raise selenium.common.exceptions.TimeoutException(msg='XPath "{}" presence wait timeout.'.format(xpath))
    return elem


def _wait_for_element_click(xpath, wait):
    try:
        polling_f = expected_conditions.element_to_be_clickable((selenium.webdriver.common.by.By.XPATH, xpath))
        elem = wait.until(polling_f)
    except: 
        raise selenium.common.exceptions.TimeoutException(msg='XPath "{}" presence wait timeout.'.format(xpath))
    return elem

In [92]:
#define short and long timeouts
wait_timeouts=(30, 180)

#open the driver (change the executable path to geckodriver_mac or geckodriver.exe)
driver = selenium.webdriver.Firefox(executable_path="./geckodriver")

#define short and long waits (for the times you have to wait for the page to load)
short_wait = selenium.webdriver.support.ui.WebDriverWait(driver, wait_timeouts[0], poll_frequency=0.05)
long_wait = selenium.webdriver.support.ui.WebDriverWait(driver, wait_timeouts[1], poll_frequency=1)


In [93]:
#get a website
driver.get("http://vds.issproxy.com/SearchPage.php?CustomerID=4372")
#let's wait until we find the fund
element = _wait_for_element('//*[@id="selFundID"]',short_wait)

In [94]:
#let's wait until we find the fund
element = _wait_for_element('//*[@id="selFundID"]',short_wait)

In [95]:
#print the possible options
for option in element.find_elements_by_xpath('//*[@id="selFundID"]/option'):
    print(option.text)

Select a fund:
CSTG&E International Social Core Equity Portfolio
CSTG&E U.S. Social Core Equity 2 Portfolio
DFA International Real Estate Securities Portfolio
DFA International Small Cap Value Portfolio
DFA International Value ex Tobacco Portfolio
DFA Real Estate Securities Portfolio
Dimensional Emerging Markets Value Fund
Emerging Markets Core Equity Portfolio
Emerging Markets Social Core Equity Portfolio
International Core Equity Portfolio
International Large Cap Growth Portfolio
International Small Cap Growth Portfolio
International Social Core Equity Portfolio
International Sustainability Core 1 Portfolio
International Vector Equity Portfolio
Large Cap International Portfolio
T.A. U.S. Core Equity 2 Portfolio
T.A. World ex U.S. Core Equity Portfolio
Tax-Managed DFA International Value Portfolio
Tax-Managed U.S. Equity Portfolio
Tax-Managed U.S. Small Cap Portfolio
Tax-Managed U.S. Targeted Value Portfolio
The Asia Pacific Small Company Series
The Canadian Small Company Series
The C

In [96]:
#keep the options
options = element.find_elements_by_xpath('//*[@id="selFundID"]/option')

In [97]:
#click on the first one
options[1].click()

In [98]:
element.find_element_by_xpath('//*[@id="SPFundInputButton"]').click()

In [101]:
companies = driver.find_elements_by_xpath('//*[@id="MPCompanyNameLink"]')

In [102]:
companies[0].text

'ABB Ltd.'

In [103]:
companies[0].click()

In [104]:
driver.back()

In [90]:
#Just to show the kind of data 
import pandas as pd

#define short and long timeouts
wait_timeouts=(2, 180)

#open the driver (change the executable path to geckodriver_mac or geckodriver.exe)
driver = selenium.webdriver.Firefox(executable_path="./geckodriver")

#define short and long waits (for the times you have to wait for the page to load)
short_wait = selenium.webdriver.support.ui.WebDriverWait(driver, wait_timeouts[0], poll_frequency=0.05)
long_wait = selenium.webdriver.support.ui.WebDriverWait(driver, wait_timeouts[1], poll_frequency=1)


#get a website
driver.get("http://vds.issproxy.com/SearchPage.php?CustomerID=4372")

#let's wait until we find the fund
element = _wait_for_element('//*[@id="selFundID"]',short_wait)

#keep the options
options = element.find_elements_by_xpath('//*[@id="selFundID"]/option')

#click on the second one
options[1].click()
element.find_element_by_xpath('//*[@id="SPFundInputButton"]').click()

_wait_for_element('//*[@id="MPCompanyNameLink"]',short_wait)
companies = driver.find_elements_by_xpath('//*[@id="MPCompanyNameLink"]')

#For each company
for i in range(len(companies)):
    #Find the companies (otherwise it's lost while refreshing the page)
    _wait_for_element('//*[@id="MPCompanyNameLink"]',short_wait)
    companies = driver.find_elements_by_xpath('//*[@id="MPCompanyNameLink"]')

    
    #Click on a company and print the name
    comp = companies[i]
    print(i,comp.text)
    time.sleep(2)
    comp.click()
    
    #let's wait until we find the the table, read it and display the header
    html = _wait_for_element('//*[@id="VPOuterContainer"]',short_wait).get_attribute('innerHTML')
    display(pd.read_html(html,header=[0,1])[0].head())
    
    #Go back to the previous screen
    driver.back()
    

0 ABB Ltd.


,#,Proposal,Vote
,Meeting for ADR Holders,Unnamed: 1_level_1,Unnamed: 2_level_1
0,1.0,Accept Financial Statements and Statutory Reports,For
1,2.0,Approve Remuneration Report,For
2,3.0,Approve Discharge of Board and Senior Management,For
3,4.0,Approve Allocation of Income,For
4,5.0,Approve CHF 86 Million Reduction in Share Capi...,For


1 Agnico Eagle Mines Limited


,#,Proposal,Vote
,Management proposals,Unnamed: 1_level_1,Unnamed: 2_level_1
0,1.1,Elect Director Leanne M. Baker,For
1,1.2,Elect Director Sean Boyd,For
2,1.3,Elect Director Martine A. Celej,For
3,1.4,Elect Director Robert J. Gemmell,For
4,1.5,Elect Director Mel Leiderman,For


2 Agrium Inc.


,#,Proposal,Vote
,Management proposals,Unnamed: 1_level_1,Unnamed: 2_level_1
0,1.1,Elect Director Maura J. Clark,For
1,1.2,Elect Director David C. Everitt,Withhold
2,1.3,Elect Director Russell K. Girling,For
3,1.4,Elect Director Russell J. Horner,For
4,1.5,Elect Director Miranda C. Hubbs,For


3 Agrium Inc.


,#,Proposal,Vote
,Management proposals,Unnamed: 1_level_1,Unnamed: 2_level_1
0,1,Approve Merger Agreement with Potash Corporati...,For


KeyboardInterrupt: 

In [89]:
companies.text

'ABB Ltd.'

- Move between windows
- Find by attributes (like in beautifulsoup)
- Run javascript or ask for javascript variables
